In [3]:
from collections import namedtuple
from itertools import accumulate
from operator import xor
from typing import Callable
import random
from copy import deepcopy
import math
import logging

In [4]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [5]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [8]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [9]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]


In [10]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

In [16]:
NUM_MATCHES = 30
NIM_SIZE = 10



def evaluate(strategy: Callable) -> float:
    opponent = (strategy, optimal_startegy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES


evaluate(make_strategy({"p": 0.9999}))

0.0

In [17]:
def human_Agent(state: Nim) -> NimPly:
    
    selectedRow = int(input('Enter the row: '))
    elementsToRemove = int(input('Enter the number of elements to remove: '))
    ply = NimPly(selectedRow, elementsToRemove)
    return ply

In [18]:
def random_Agent(state: Nim) -> NimPly:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return NimPly(row, num_objects)

In [19]:
def evolvedRules(genome: list) -> Callable:
    def evolvable(state: Nim) -> NimPly:
        data = cookStatus(state)
        if random.random() < genome[0] and data["activeRowsNumber"] % 2 == 0:
            if random.random() < genome[1]:
                row = data["longestRow"]
            elif random.random() < genome[2]:
                row = data["shortestRow"]
            else:
                row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
        else:
            if random.random() < genome[3]:
                row = data["longestRow"]
            elif random.random() < genome[4]:
                row = data["shortestRow"]
            else:
                row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
        if random.random() < genome[5] and data["totalElements"] % 2 == 0:
            if random.random() < genome[6]:
                
                num_objects = state.rows[row] if (state.k is None or state.rows[row] <= state.k) else state.k
            elif random.random() < genome[7]:
                
                num_objects = state.rows[row] - 1 if (state.k is None or state.rows[row] - 1 <= state.k) else state.k
            else:
                num_objects = math.floor(state.rows[row] * genome[8])
        else:
            if random.random() < genome[9]:
                
                num_objects = state.rows[row] if (state.k is None or state.rows[row] <= state.k) else state.k
            elif random.random() < genome[10]:
                
                num_objects = state.rows[row] - 1 if (state.k is None or state.rows[row] - 1 <= state.k) else state.k
            else:
                num_objects = math.floor(state.rows[row] * genome[11])
        
        return NimPly(row, num_objects)

    return evolvable        

In [20]:
def GA(populationSize: int, offspringSize: int, numGenerations: int) -> list:
    
    Individual = namedtuple('Individual', ['genome', 'fitness'])
    population = list()

    def tournament(population: list) -> Individual:
        return max(random.choices(population, k=2), key=lambda i: i.fitness)
    

    def crossOver(genome1: list, genome2: list) -> list:
        cut = random.randint(0, len(genome1))
        return genome1[:cut] + genome2[cut:]


    def mutation(genome: list) -> list:
        point = random.randint(0, len(genome) - 1)
        if random.random() < 0.5:
            genome[point] = genome[point] + 0.1 if genome[point] < 0.9 else genome[point] - 0.5
        else:
            genome[point] = genome[point] - 0.1 if genome[point] > 0.1 else genome[point] + 0.5
        return genome


    for genome in [[round(random.random(), 3) for _ in range(12)] for _ in range(populationSize)]:
        fitness = evaluate(evolvedRules(genome), randomAgent)
        population.append(Individual(genome, fitness))

    for g in range(numGenerations):
        offspring = list()
        
        for i in range(offspringSize):
            if random.random() < 0.5:
                p = tournament(population)
                o = mutation(p.genome)
                f = evaluate(evolvedRules(o), randomAgent)
                offspring.append(Individual(o, f))
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = crossOver(p1.genome, p2.genome)
                f = evaluate(evolvedRules(o), randomAgent)
                offspring.append(Individual(o, f))
        
        population += offspring
        population = sorted(population, key=lambda i: -i.fitness)[:populationSize]

    return population[0].genome


def evaluate(agent1: Callable, agent2: Callable) -> float:
    
    match = (agent1, agent2)
    won = 0
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = match[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES


def simulate(agent1: Callable, agent2: Callable) -> None:
    match = (agent1, agent2)
    nim = Nim(NIM_SIZE)
    print(f'status: Initial board  -> {nim}')
    player = 0
    while nim:
        ply = match[player](nim)
        nim.nimming(ply)
        print(f'status: After {match[player].__name__} -> {nim}')
        player = 1 - player
    print(f"status: {match[player].__name__} won!")


In [22]:



logging.getLogger().setLevel(logging.DEBUG)

strategy = (make_strategy({"p": 0.1}), optimal_startegy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 20>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 13 15 7 19 20>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 7 19 14>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 13 15 7 9 14>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 3 7 9 14>
DEBUG:root:status: After player 1 -> <1 3 5 7 5 11 13 3 7 9 14>
DEBUG:root:status: After player 0 -> <1 3 5 7 5 11 13 3 7 9 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 5 7 13 3 7 9 2>
DEBUG:root:status: After player 0 -> <1 3 5 7 5 7 3 3 7 9 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 5 7 3 3 7 7 2>
DEBUG:root:status: After player 0 -> <0 3 5 7 5 7 3 3 7 7 2>
DEBUG:root:status: After player 1 -> <0 2 5 7 5 7 3 3 7 7 2>
DEBUG:root:status: After player 0 -> <0 2 5 2 5 7 3 3 7 7 2>
DEBUG:root:status: After player 1 -> <0 2 0 2 5 7 3 3 7 7 2>
DEBUG:root:status: After player 0 -> <0 2 0 2 5 4 